In [1]:
from functools import reduce
import statistics as st
import cv2
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#import matplotlib.pyplot as plt
import os
#from PyPDF2 import PdfReader
#%matplotlib inline
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\epenago\AppData\Local\Tesseract-OCR\tesseract.exe'

In [2]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

In [3]:
def calcular_espacios(diccionario, indices):
    
    suma = 0
    bandera = True
  
    width_espacios = []
    for i in range(len(indices)):
             
        if(bandera == True):
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            if i < len(indices)-1:
                if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                    pass
                else:
                    bandera = True
            else:           
                    bandera = True
                    

        elif(i < len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                pass
            else:
                bandera = True
               

        elif(i <= len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            bandera = True
            
    #print(width_espacios)
    width_espacios.sort() 
    
    moda = st.mode(width_espacios)
    return (moda-(moda*1), moda+(moda*1))

In [4]:
def coordenadas_extraccion(diccionario, indices, intervalo):
    
    datos_extraidos = []
    dict = {}
    coordenadas = []
    suma = 0
    bandera = True
    indice_anterior = -1
    strings = []
    string = ""
    index = []
    line = 0

    for i in range(len(indices)):
             
        if(bandera == True):
            indice_anterior = i
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            # Se guarda la palabra y su indice
            string = diccionario["text"][indices[i]]
            index.append(indices[i])
            
            if i < len(indices)-1:
                if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                    
                    if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                        bandera = True
                        #print(indices[i])
                        coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                        diccionario["top"][indices[indice_anterior]], 
                        suma,
                        diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                        
                        dict["text"] = string
                        dict["line"] = line
                        dict["index"] = index
                        dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                        datos_extraidos.append(dict)
                                                                    
                        dict = {}                        
                        string = ""
                        index = []

                        bandera = True
                        indice_anterior = -1
                else:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                    

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []
                    line += 1

                    bandera = True
                    indice_anterior = -1
            else:           
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1

        elif(i < len(indices)-1):
            string += (" " + diccionario["text"][indices[i]])
            index.append(indices[i])

            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                
                if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1
            else:
                bandera = True
                #print(indices[i])
                coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                diccionario["top"][indices[indice_anterior]], 
                suma,
                diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                dict["text"] = string
                dict["line"] = line
                dict["index"] = index
                dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                datos_extraidos.append(dict)
                                                            
                dict = {}                        
                string = ""
                index = []
                line += 1

                bandera = True
                indice_anterior = -1

        elif(i <= len(indices)-1):
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            coordenadas.append([diccionario["left"][indices[indice_anterior]], 
            diccionario["top"][indices[indice_anterior]], 
            suma,
            diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

            string = ""
            index = []

            bandera = True
            indice_anterior = -1

    return (coordenadas, datos_extraidos)

En el Metodo extraccion_columns logramos separar las columnas extrayedo y clasificando los indices en el dicionario data_ordenaizer

In [5]:
def extracion_colums(df):

    arr = np.array(df["coordinates"].to_list())
    colums = []
    
    for i in np.arange(len(arr)):
        temp = []
        temp.append(i)
        for j in np.arange(len(arr)):

            if i == j: continue
            interval1 = np.arange(arr[i,0], arr[i,1]+1, dtype=int)
            interval2 = np.arange(arr[j,0], arr[j,1]+1, dtype=int)

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 55 and percent2 > 55:
                temp.append(j)
        
        colums.append(temp)
        
    cl = []
    descartados = []
    for i in np.arange(len(colums)):
        temp = []
        temp = colums[i]
        if i in descartados: continue
        for j in np.arange(len(colums)):

            if i == j: continue
            interval1 = colums[i]
            interval2 = colums[j]

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 20 or percent2 > 20:
                temp += colums[j]
                descartados.append(j)
        cl.append(temp)

    colums = []    
    #se eliminan elementos repetidos en las columnas
    for element in cl:
        temp = []
        for i in element:
            if i not in temp:
                temp.append(i)
        colums.append(temp)
    

    print("el tamaño de cl es: ", len(cl))
    #en este bloque se saca la pocision en pixeles de la imagen
    min_max = []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max.append((menor,mayor))

    for i in range(len(min_max)):
        print(min_max[i], colums[i])

    cl = []
    descartados = []
    by_delete = []
    for i in range(len(min_max)):
        temp= []
        temp= colums[i]
        if i in descartados: continue
        for j in range(len(min_max)):

            if i==j: continue
            if min_max[i][0]-10 <= min_max[j][0] and min_max[i][1]+10 >= min_max[j][1]:
                temp += colums[j]
                descartados.append(j)
                if i > j:
                    by_delete.append(colums[j])
        cl.append(temp)

    colums = cl
    print("estos son a eliminar", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    print()
    zises = []
    for element in colums:
        print(element)
        zises.append(len(element))
    
    mean = st.mean(zises)
    print("la media de los len ", mean)
    by_delete = []
    for i in range(len(colums)):
        percent20 = (mean/100)*20
        #print("percent20: ", percent20)
        if len(colums[i]) < percent20:
            by_delete.append(colums[i])

    for element in colums:
        print(element)
        print

    print("elementos a eliminar: ", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    for element in colums:
        print(element)

    min_max_h= []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max_h.append((menor,mayor))

    min_max_h_unOrder = min_max_h.copy()
    min_max_h.sort()

    cl = []
    for i in np.arange(len(colums)):
        for j in np.arange(len(colums)):
            if min_max_h_unOrder[j] == min_max_h[i]:
                cl.append(colums[j])

    colums = cl

    print(min_max_h)
    
    return min_max_h,colums

El metodo build_df toma el arreglo con las columnas separadar y utiliza el diccionario construido en extracion_colums 
para crear un objeto tipo pandas.DataFrame con la misma distribucion de los datos que estaba en la imagen

In [6]:
def isNumeric(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [7]:
def build_df(pd_dataOrdenizer, colums):
    temp = pd.DataFrame()
    pd_columns = pd.DataFrame()
    for i in range(len(colums)):
        temp = pd.DataFrame.copy(pd_dataOrdenizer.loc[colums[i], ["text", "line"]])
        temp = temp.drop_duplicates()
        temp = temp.sort_values("line")
        temp = pd.Series(temp["text"].tolist(), index=temp["line"].tolist())
        temp = temp.loc[~temp.index.duplicated(keep='first')]
        pd_columns = pd.concat([pd_columns, temp], axis=1, ignore_index=True)

    pd_columns = pd_columns.sort_index()
    return pd_columns

In [8]:
def del_headEnd(df):

    freq = df['line'].value_counts().sort_index().head(10)
    to_delete = []
    bandera = True

    for i in range(len(freq)):
        if bandera == True:
            if freq[i] == 1:
                to_delete.append(i)
            
            elif freq[i] == 2 and freq[i+1] == 1:
                to_delete.append(i)
            else: bandera = False
    
    for i in to_delete:
        df.drop(df.index[df['line'] == i] , axis=0, inplace=True)
    df.reset_index(inplace=True)
    df.drop(['level_0'], axis=1, inplace=True)
    

In [203]:
#main
#path = r'./reto/1.PNG'
path = r'.\reto\14.PNG'
image = cv2.imread(path)
gray = get_grayscale(image)
th = thresholding(gray)
openingg = opening(gray)
cannyy = canny(gray)
img = image
hImg, wImg, _ = img.shape

custom_config = r'-c tessedit_char_blacklist=$¢§_{} --oem 3 --psm 4' 
data = pytesseract.image_to_data(img, lang= 'eng+spa', config=custom_config, output_type= 'dict')


In [204]:
pd.DataFrame(data).tail(2)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
471,5,1,14,1,1,9,312,1540,44,10,86.683586,estados
472,5,1,14,1,1,10,360,1540,62,10,94.773262,financieros


In [205]:
indice_words = []
for i in range(len(data['text'])):
    if data['text'][i] != '' and data['text'][i] != ' ' and float(data['conf'][i]) >= 50 and data["text"][i] != "-" and data["text"][i] != "--" and data["text"][i] != "s" and data["text"] != "S":
        indice_words.append(i)

#print(len(indice_words))
#print(indice_words)
intervalo = calcular_espacios(data, indice_words)
print(intervalo)
coordenadas,data_organizer = coordenadas_extraccion(data,indice_words, intervalo)
df_data_organizer = pd.DataFrame(data_organizer)

(0, 8)


In [206]:
df_data_organizer.head(10)

,text,line,index,coordinates
0,Notas,0,[4],"(522, 567)"
1,31/12/2020,0,[5],"(615, 695)"
2,31/12/2019,0,[6],"(786, 866)"
3,ACTIVO,1,[46],"(23, 71)"
4,ACTIVOS CORRIENTES:,2,"[48, 49]","(23, 164)"
5,Efectivo y equivalentes al efectivo,3,"[51, 52, 53, 54, 55]","(33, 223)"
6,2,3,[56],"(531, 538)"
7,Us,3,[57],"(580, 587)"
8,"249,587,111",3,[58],"(650, 720)"
9,"330,155,679",3,[60],"(821, 892)"


In [207]:

df_data_organizer.loc[:,"coordinates"].apply(lambda x: (x[0]+x[1])/2).head(10)

0    544.5
1    655.0
2    826.0
3     47.0
4     93.5
5    128.0
6    534.5
7    583.5
8    685.0
9    856.5
Name: coordinates, dtype: float64

In [208]:
del_headEnd(df_data_organizer)

In [209]:
def isNumericColum(df): 
    indexs=[]
    for i in df.columns:
        size = df[i].apply(isNumeric).value_counts().sum()
        try:
            if 100*df[i].apply(isNumeric).value_counts()[True]/size >= 80:
                indexs.append('num')
            else:
                indexs.append('text')
        except KeyError:
            indexs.append('text')
    df.columns = indexs


In [210]:
df_data_organizer.head(5)

,text,line,index,coordinates
0,Notas,0,[4],"(522, 567)"
1,31/12/2020,0,[5],"(615, 695)"
2,31/12/2019,0,[6],"(786, 866)"
3,ACTIVO,1,[46],"(23, 71)"
4,ACTIVOS CORRIENTES:,2,"[48, 49]","(23, 164)"


In [211]:


#print("numero de posiciones: " + str(len(coordenadas)))
#del_headEnd(df_data_organizer)
min_max_h,colums = extracion_colums(df_data_organizer)

#Dibujado de los rectangulos en la imagen
for p in min_max_h:
    cv2.rectangle(img, (p[0], hImg-5),(p[1], 5), (random.randint(0,255), random.randint(0,255), random.randint(0,255)), 1)

for p in coordenadas:
    cv2.rectangle(img, (p[0], p[1]),(p[2], p[3]), (50, 50, 255), 1)

cv2.imwrite('salida.png',img)

df = pd.DataFrame(build_df(df_data_organizer , colums))
df.to_excel("data.xlsx", index=False)

"""
img = cv2.resize(img, (600, 700))
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

el tamaño de cl es:  9
(522, 567) [0]
(615, 721) [1, 8, 13, 23, 52, 63, 75, 93, 108, 116, 18, 26, 29, 38, 42, 58, 67, 70, 79, 84, 87, 90, 102, 105]
(786, 892) [2, 9, 14, 24, 39, 53, 64, 76, 94, 97, 109, 111, 117, 121, 19, 27, 43, 48, 60, 68, 71, 82, 85, 91, 100, 103, 106, 113]
(23, 430) [3, 20, 72, 104, 44, 45, 46, 54, 78, 89, 95, 98, 99, 28, 4, 5, 17, 25, 30, 31, 33, 34, 35, 36, 40, 49, 50, 55, 56, 57, 66, 69, 73, 74, 77, 81, 88, 92, 101, 114, 118, 15, 83, 86, 32, 10, 61, 65, 80, 107, 110, 112]
(23, 430) [4, 5, 17, 25, 28, 30, 31, 33, 34, 35, 36, 40, 44, 45, 46, 49, 50, 54, 55, 56, 57, 66, 69, 73, 74, 77, 81, 88, 89, 92, 95, 98, 99, 101, 114, 118, 3, 20, 72, 104, 78, 15, 83, 86, 32, 10, 61, 65, 80, 107, 110, 112]
(531, 538) [6, 11, 21, 62]
(580, 588) [7, 12, 22, 37, 96, 115, 119]
(570, 588) [16, 41, 47, 51]
(751, 759) [59, 120]
estos son a eliminar []

[0, 6, 11, 21, 62]
[1, 8, 13, 23, 52, 63, 75, 93, 108, 116, 18, 26, 29, 38, 42, 58, 67, 70, 79, 84, 87, 90, 102, 105]
[2, 9, 14, 24, 3

"\nimg = cv2.resize(img, (600, 700))\ncv2.imshow('Result', img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [212]:
for i in df.columns:
    df[i] = df.apply(lambda x: x[i].replace(',','') if type(x[i]) == str else x[i], axis=1)
    df[i] = df.apply(lambda x: x[i].replace('.','') if type(x[i]) == str else x[i], axis=1)
    df[i] = df.apply(lambda x: '0' if x[i] == 'o' else x[i], axis=1)
isNumericColum(df)


In [213]:
df

,text,text,num,num
0,NaN,NaN,31/12/2020,31/12/2019
1,ACTIVO,NaN,NaN,NaN
2,ACTIVOS CORRIENTES:,NaN,NaN,NaN
3,Efectivo y equivalentes al efectivo,Us,249587111,330155679
4,Cuentas comerciales por cobrar y otras cuentas...,Us,891927450,492729271
5,Arrendamiento financiero por Cobrar corriente,”s,NaN,NaN
6,Inventarios,Us,376027233,251188733
7,Gastos pagados por anticipado corrientes,NaN,88376797,78319364
8,Otros activos corrientes,NaN,46680490,NaN
9,Total activos corrientes,NaN,NaN,NaN
